In [1]:
import json
import string
import random
import re
import urllib.request
from nltk.stem import WordNetLemmatizer
import numpy as np
# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs
import nltk
# Descargar el diccionario
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sys
#import gradio as gr
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import text_to_word_sequence

import multiprocessing
from gensim.models import Word2Vec

In [2]:
!{sys.executable} -m pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.5 MB/s eta 0:

In [3]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [4]:
def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [5]:
def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [6]:
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


##TOPICS:
* HOT DOG
* FAST FOOD
* COCA COLA
* FOOD
* OBESITY
* OVERWEIGHT
* SEDENTARY LIFESTYLE

##WE ONLY SELECT 3 TOPICS RANDOMLY, IT CHANGES IN EACH EXECUTION

In [7]:
topics = ["https://en.wikipedia.org/wiki/Hot_dog","https://en.wikipedia.org/wiki/Fast_food","https://en.wikipedia.org/wiki/Coca-Cola","https://en.wikipedia.org/wiki/Food","https://en.wikipedia.org/wiki/Obesity","https://en.wikipedia.org/wiki/Overweight","https://en.wikipedia.org/wiki/Sedentary_lifestyle"]

full_article = ""

for i in topics:
  raw_html = urllib.request.urlopen(i)
  raw_html = raw_html.read()

  article_html = bs.BeautifulSoup(raw_html, 'lxml')

  article_paragraphs = article_html.find_all('p')

  article_text = ''

  for para in article_paragraphs:
      article_text += para.text

  article_text = article_text.lower()
  full_article = full_article + article_text

In [8]:
text = re.sub(r'\[[0-9]*\]', ' ', full_article)
text = re.sub(r'\s+', ' ', text)

In [9]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [10]:
option = 0

In [11]:
corpus_1 = [s.split() for s in corpus]

corpus_2 = []
for t in corpus:
    corpus_2.append(text_to_word_sequence(t))

In [13]:
# Crearmos el modelo generador de vectoeres

w2v_model_1 = Word2Vec(
                     min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram


w2v_model_2 = Word2Vec(
                     min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [14]:
w2v_model_1.build_vocab(corpus_1)
w2v_model_2.build_vocab(corpus_2)

In [15]:
print("Cantidad de docs en el corpus:", w2v_model_1.corpus_count)
print("Cantidad de docs en el corpus:", w2v_model_2.corpus_count)

Cantidad de docs en el corpus: 1188
Cantidad de docs en el corpus: 1188


In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_1.train(corpus,
                 total_examples=w2v_model_1.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

w2v_model_2.train(corpus,
                 total_examples=w2v_model_2.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 12209.306640625
Loss after epoch 1: 5655.638671875
Loss after epoch 2: 5653.0703125
Loss after epoch 3: 5229.556640625
Loss after epoch 4: 4999.955078125
Loss after epoch 5: 5290.69140625
Loss after epoch 6: 5276.94140625
Loss after epoch 7: 5086.84765625
Loss after epoch 8: 4857.09375
Loss after epoch 9: 5356.12109375
Loss after epoch 10: 5142.3359375
Loss after epoch 11: 5086.58984375
Loss after epoch 12: 5187.3203125
Loss after epoch 13: 4556.4453125
Loss after epoch 14: 5044.78125
Loss after epoch 15: 5073.96875
Loss after epoch 16: 4744.53125
Loss after epoch 17: 4442.6953125
Loss after epoch 18: 4969.6171875
Loss after epoch 19: 4626.359375
Loss after epoch 0: 48303.71484375
Loss after epoch 1: 31027.69140625
Loss after epoch 2: 29698.7890625
Loss after epoch 3: 29268.1015625
Loss after epoch 4: 29241.984375
Loss after epoch 5: 28694.109375
Loss after epoch 6: 28768.125
Loss after epoch 7: 28522.4375
Loss after epoch 8: 28658.703125
Loss after epoch 9: 28395.1

(1304508, 3408000)

##APARCIÓN DEL BMI, BODY MASS INDEX

In [17]:
word_p = ["healthy"]

In [18]:
w2v_model_1.wv.most_similar(positive=word_p,topn=10)

[('group', 0.1414330005645752),
 ('held', 0.1382761001586914),
 ('consumers', 0.13777770102024078),
 ('class', 0.13561250269412994),
 ('thus', 0.13519111275672913),
 ('children.', 0.13360245525836945),
 ('rate', 0.1331871896982193),
 ('advertisements', 0.1307992786169052),
 ('company,', 0.12945395708084106),
 ('received', 0.1279081404209137)]

In [19]:
w2v_model_2.wv.most_similar(positive=word_p,topn=10)

[('significantly', 0.18254433572292328),
 ('negative', 0.16583077609539032),
 ('known', 0.15238207578659058),
 ('treatments', 0.1516413539648056),
 ('served', 0.1463031768798828),
 ('if', 0.14583048224449158),
 ('000', 0.14335739612579346),
 ('contains', 0.14265966415405273),
 ('nuts', 0.1391775757074356),
 ('syndrome', 0.13550493121147156)]

In [20]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model_1)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

* Para 0<Y<10 y, -10<X<-5 vemos que fast, food y coke están agrupados, claramente existe relación, podemos relacionar coke(Coca-Cola) con food o fast food

* Para -13<Y<-10 y, -11<X<-10 vemos que national y countries tienen cercania, como weight, medical y fast

*  Para -5<Y<-4 y, 13.5<X<14 tenemos lifestyle y foods, claramente relación entre ambas palabras

El modelo genera relaciones entre palabras claves mas recurrentes que otras, a diferencia de stop words o palabras complementarias de una oración